# Model Evaluation

Process 
1. Import data & set up dummy variables
    * drop 1 column for each grouping!
2. Split data: 
    * Train / Test: X, X_test, y, y_test (~20%)
    * Train / Val: X_train, X_val, y_train, y_val (~60%?)
3. Create 3 Models: Linear, Ridge, Lasso
    * SCALE: Ridge & Lasso
    * Keep default alphas
    * use KFolds
4. Evaluate basic models on scoring (r2 train, r2 val)
    * Use KFolds 
    * if models are really close to one another, tune alphas
    * Pick an alpha range to loop through and compare results 
5. Tune alphas for Ridge & Lasso
    * difference between r2 train and r2 val will be guide
    * if r2 train > r2 val --> overfitting --> increase alpha
6. Pick best version of each model to evaluate
    * linear
    * ridge at best alpha
    * lasso at best alpha
7. RETRAIN best model on entire training data (X)
8. Evaluate model against hold out data (predict x_test vs. y_test) 
    * r2 train versus r2 test
    * if difference is significant, more model tuning

In [1]:
# Necessary imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, lars_path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

sns.set(context='notebook', style='whitegrid', font_scale=1.2)

## Data Import & Setup

In [2]:
sf = pd.read_csv('data_files/sf_clean.csv')

# Reformatting hood_district names
sf['hood_district'] = sf['hood_district'].astype(int)
sf['hood_district'] = sf['hood_district'].astype(str)

In [3]:
sf = pd.get_dummies(sf)

In [4]:
sf.columns

Index(['price', 'sqft', 'beds', 'bath', 'laundry_(a) in-unit',
       'laundry_(b) on-site', 'laundry_(c) no laundry', 'pets_(a) both',
       'pets_(b) dogs', 'pets_(c) cats', 'pets_(d) no pets',
       'housing_type_(a) single', 'housing_type_(b) double',
       'housing_type_(c) multi', 'parking_(a) valet', 'parking_(b) protected',
       'parking_(c) off-street', 'parking_(d) no parking', 'hood_district_1',
       'hood_district_10', 'hood_district_2', 'hood_district_3',
       'hood_district_4', 'hood_district_5', 'hood_district_6',
       'hood_district_7', 'hood_district_8', 'hood_district_9'],
      dtype='object')

In [5]:
cols_to_drop = ['laundry_(c) no laundry',
               'pets_(d) no pets', 
               'housing_type_(c) multi',
               'parking_(d) no parking',
               'hood_district_10']

sf = sf.drop(cols_to_drop, axis=1)

In [6]:
sf.head()

,price,sqft,beds,bath,laundry_(a) in-unit,laundry_(b) on-site,pets_(a) both,pets_(b) dogs,pets_(c) cats,housing_type_(a) single,...,parking_(c) off-street,hood_district_1,hood_district_2,hood_district_3,hood_district_4,hood_district_5,hood_district_6,hood_district_7,hood_district_8,hood_district_9
0,6800,1600.0,2.0,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3500,550.0,1.0,1.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,5100,1300.0,2.0,1.0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,9000,3500.0,3.0,2.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3100,561.0,1.0,1.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Training & Hold-Out Splits

### Train / Test

In [7]:
X, y = sf.drop('price',axis=1), sf['price']

# hold out 20% of the data for final testing
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10)

### Train / Validation

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=10)

## Models

### Linear

In [9]:
lin = LinearRegression()

lin.fit(X_train, y_train)
print(f'Linear Regression train R^2: {lin.score(X_train, y_train):.5f}')
print(f'Linear Regression val R^2: {lin.score(X_val, y_val):.5f}')

Linear Regression train R^2: 0.78184
Linear Regression val R^2: 0.67066


### Ridge

#### Scaling Features for Lasso / Ridge

In [10]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

#### Model Setup

In [11]:
rid = Ridge()

rid.fit(X_train_scaled, y_train)
print(f'Ridge Regression test R^2: {rid.score(X_train_scaled, y_train):.5f}')
print(f'Ridge Regression val R^2: {rid.score(X_val_scaled, y_val):.5f}')

Ridge Regression test R^2: 0.78181
Ridge Regression val R^2: 0.67054


### Lasso

In [12]:
las = Lasso()

las.fit(X_train_scaled, y_train)
print(f'Ridge Regression test R^2: {las.score(X_train_scaled, y_train):.5f}')
print(f'Ridge Regression val R^2: {las.score(X_val_scaled, y_val):.5f}')

Ridge Regression test R^2: 0.78172
Ridge Regression val R^2: 0.67067


### Model Evaluation using KFolds

In [13]:
sf2 = sf.copy()

X, y = sf2.drop('price',axis=1), sf['price']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=30)

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state =12)

cv_lin_r2s, cv_rid_r2s, cv_las_r2s = [], [], []  #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lin = LinearRegression()
    rid = Ridge()
    las = Lasso()

    lin.fit(X_train, y_train)
    cv_lin_r2s.append(lin.score(X_val, y_val))
    
    #ridge/lasso with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    rid.fit(X_train_scaled, y_train)
    cv_rid_r2s.append(rid.score(X_val_scaled, y_val))
    
    
    #lasso with feature scaling
    las.fit(X_train_scaled, y_train)
    cv_las_r2s.append(las.score(X_val_scaled, y_val))

print('Simple scores: ', cv_lin_r2s)
print('Ridge scores: ', cv_rid_r2s)
print('Lasso scores: ', cv_las_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lin_r2s):.5f} +- {np.std(cv_lin_r2s):.5f}')
print(f'Ridge mean cv r^2: {np.mean(cv_rid_r2s):.5f} +- {np.std(cv_rid_r2s):.5f}')
print(f'Lasso mean cv r^2: {np.mean(cv_las_r2s):.5f} +- {np.std(cv_las_r2s):.5f}')

Simple scores:  [0.7902121151705227, 0.678231918470749, 0.6539934339757945, 0.7160366020304101, 0.8024668639219245]
Ridge scores:  [0.789965695329414, 0.6794760528664688, 0.6538530428786878, 0.7162344132034061, 0.8022393400128285]
Lasso scores:  [0.7904453069834163, 0.6802749348235684, 0.6541668011093027, 0.7158473766765832, 0.8020956988709493] 

Simple mean cv r^2: 0.72819 +- 0.05918
Ridge mean cv r^2: 0.72835 +- 0.05889
Lasso mean cv r^2: 0.72857 +- 0.05876


## Alpha Tuning - Ridge & Lasso

In [15]:
sf3 = sf.copy()

X, y = sf3.drop('price',axis=1), sf['price']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=30)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=10)

In [17]:
# Scalilng features
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

### Lasso

In [18]:
alphavec = 10**np.linspace(-2,2,500)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-02, 1.01862899e-02, 1.03760502e-02, 1.05693455e-02,
       1.07662418e-02, 1.09668060e-02, 1.11711065e-02, 1.13792129e-02,
       1.15911962e-02, 1.18071285e-02, 1.20270833e-02, 1.22511358e-02,
       1.24793621e-02, 1.27118400e-02, 1.29486487e-02, 1.31898690e-02,
       1.34355829e-02, 1.36858742e-02, 1.39408283e-02, 1.42005318e-02,
       1.44650734e-02, 1.47345431e-0...
       6.54080591e+01, 6.66265452e+01, 6.78677305e+01, 6.91320378e+01,
       7.04198979e+01, 7.17317494e+01, 7.30680395e+01, 7.44292233e+01,
       7.58157646e+01, 7.72281357e+01, 7.86668179e+01, 8.01323013e+01,
       8.16250851e+01, 8.31456781e+01, 8.46945981e+01, 8.62723729e+01,
       8.78795401e+01, 8.95166472e+01, 9.11842520e+01, 9.28829225e+01,
       9.46132376e+01, 9.63757866e+01, 9.81711702e+01, 1.00000000e+02]),
        cv=5)

In [19]:
las_alpha = lasso_model.alpha_
las_alpha

2.1117830444482424

In [20]:
las = Lasso(alpha=las_alpha)

las.fit(X_train_scaled, y_train)
print(f'Lasso Regression test R^2: {las.score(X_train_scaled, y_train):.5f}')
print(f'Lasso Regression val R^2: {las.score(X_val_scaled, y_val):.5f}')

Lasso Regression test R^2: 0.73848
Lasso Regression val R^2: 0.78571


### Ridge

In [21]:
alphavec = 10**np.linspace(-2,2,500)

ridge_model = RidgeCV(alphas = alphavec, cv=5)
ridge_model.fit(X_train_scaled, y_train)

RidgeCV(alphas=array([1.00000000e-02, 1.01862899e-02, 1.03760502e-02, 1.05693455e-02,
       1.07662418e-02, 1.09668060e-02, 1.11711065e-02, 1.13792129e-02,
       1.15911962e-02, 1.18071285e-02, 1.20270833e-02, 1.22511358e-02,
       1.24793621e-02, 1.27118400e-02, 1.29486487e-02, 1.31898690e-02,
       1.34355829e-02, 1.36858742e-02, 1.39408283e-02, 1.42005318e-02,
       1.44650734e-02, 1.47345431e-0...
       6.54080591e+01, 6.66265452e+01, 6.78677305e+01, 6.91320378e+01,
       7.04198979e+01, 7.17317494e+01, 7.30680395e+01, 7.44292233e+01,
       7.58157646e+01, 7.72281357e+01, 7.86668179e+01, 8.01323013e+01,
       8.16250851e+01, 8.31456781e+01, 8.46945981e+01, 8.62723729e+01,
       8.78795401e+01, 8.95166472e+01, 9.11842520e+01, 9.28829225e+01,
       9.46132376e+01, 9.63757866e+01, 9.81711702e+01, 1.00000000e+02]),
        cv=5)

In [22]:
rid_alpha = ridge_model.alpha_
rid_alpha

2.5871740785959214

In [23]:
rid = Ridge(alpha=rid_alpha)

rid.fit(X_train_scaled, y_train)
print(f'Ridge Regression test R^2: {rid.score(X_train_scaled, y_train):.5f}')
print(f'Ridge Regression val R^2: {rid.score(X_val_scaled, y_val):.5f}')

Ridge Regression test R^2: 0.73873
Ridge Regression val R^2: 0.78592


In [24]:
lin_coefs = lin.coef_
rid_coefs = rid.coef_
las_coefs = las.coef_

features = sf2.columns[1:]

coef_data = {'Features': features,
             'Linear': lin_coefs,
             'Ridge': rid_coefs,
             'Lasso': las_coefs}

coef_df = pd.DataFrame(coef_data)
coef_df

,Features,Linear,Ridge,Lasso
0,sqft,2.033856,957.181236,968.179587
1,beds,114.780222,111.329017,104.323032
2,bath,445.059762,245.145479,242.251573
3,laundry_(a) in-unit,173.740595,126.507968,117.193070
4,laundry_(b) on-site,-80.711853,13.899650,6.698238
5,pets_(a) both,-34.860571,-8.201269,-4.588684
6,pets_(b) dogs,130.004859,25.407144,24.673251
7,pets_(c) cats,-31.840035,-13.327121,-11.043984
8,housing_type_(a) single,68.667593,44.934480,43.016688
9,housing_type_(b) double,62.693603,17.423840,14.759738


### Full Train & Evaluation

In [25]:
rid = Ridge(alpha=rid_alpha)

X_scaled = scaler.fit_transform(X)

rid.fit(X_train_scaled, y_train)
print(f'Ridge Regression full-train R^2: {rid.score(X_scaled, y):.5f}')
print(f'Ridge Regression test R^2: {rid.score(X_test_scaled, y_test):.5f}')

Ridge Regression full-train R^2: 0.74593
Ridge Regression test R^2: 0.82500


In [27]:
las = Lasso(alpha=las_alpha)

X_scaled = scaler.fit_transform(X)

las.fit(X_train_scaled, y_train)
print(f'Lasso Regression full-train R^2: {las.score(X_scaled, y):.5f}')
print(f'Lasso Regression test R^2: {las.score(X_test_scaled, y_test):.5f}')

Lasso Regression full-train R^2: 0.74572
Lasso Regression test R^2: 0.82481
